In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano
import theano.tensor as tt
import arviz as az
import pandas as pd
from theano import shared
from scipy import stats
import math

# Helper functions

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def normalize(x):
    return x/np.sum(x)

def rmse(pred, actual):
    return math.sqrt(np.mean((pred-actual)**2))

def sort_category(unique, count):
    output = np.ones(3) - 1
    for i in range(3):
        for j in range(len(unique)):
            if(i==unique[j]):
                output[i]=count[j]
    return normalize(output) 

def perturb(x, noise=.001):
    return tt.switch(tt.ge(x,1),1-noise, tt.switch(tt.le(x,0), noise, x))

def compute_ci_discrete(x, HDI, n, sim):
    output = [None]*sim
    for i in range(sim):
        draw = np.random.choice(x, size = n, replace = True)
        unique, counts = np.unique(draw, return_counts=True)
        output[i] = sort_category(unique, counts)
    lower = np.ones(len(output[0]))
    upper = np.ones(len(output[0]))
    output = np.transpose(output) # each element is now all the samples of one category
    for i in range(len(output)):
        lower[i] = np.quantile(output[i], q = (1-HDI)/2)
        upper[i] = np.quantile(output[i], q = 1-(1-HDI)/2)
    return [lower, upper]

def compute_ci_continuous(x, HDI, n, sim):
    output = np.ones(sim)
    for i in range(sim):
        draw = np.random.choice(x, size = n, replace = True)
        output[i] = np.mean(draw)
    lower = np.quantile(output, q = (1-HDI)/2)
    upper = np.quantile(output, q = 1-(1-HDI)/2)
    return [lower, upper]

def computeA(m, s):
    return (m*s)/(1-m)

# Computational models for non-tracking study

## I/O preprocessing

In [3]:
data = pd.read_csv('data/ecl_dat_multiple-cause.csv', encoding="ISO-8859-1")
emotion = data.loc[data['condition'] == 'emotion']
noemotion = data.loc[data['condition'] == 'no-emotion']

### Means for causal inference

In [4]:
world1human_emotion = np.array(emotion["relationBlueNorm"])
world2human_emotion = np.array(emotion["relationOrangeNorm"])
world3human_emotion = np.array(emotion["relationOrangeBlueNorm"])
worldhuman_emotion = sort_category([0,1,2],[np.mean(world1human_emotion),np.mean(world2human_emotion),np.mean(world3human_emotion)])

world1human_noemotion = np.array(noemotion["relationBlueNorm"]) 
world2human_noemotion = np.array(noemotion["relationOrangeNorm"]) 
world3human_noemotion = np.array(noemotion["relationOrangeBlueNorm"])
worldhuman_noemotion = sort_category([0,1,2],[np.mean(world1human_noemotion),np.mean(world2human_noemotion),np.mean(world3human_noemotion)])

### Means for belief inference

In [5]:
belief1human_emotion = np.array(emotion["expBlueNorm"]) 
belief2human_emotion = np.array(emotion["expOrangeNorm"]) 
belief3human_emotion = np.array(emotion["expBothNorm"]) 
beliefhuman_emotion = sort_category([0,1,2],[np.mean(belief1human_emotion),np.mean(belief2human_emotion),np.mean(belief3human_emotion)])

belief1human_noemotion = np.array(noemotion["expBlueNorm"]) 
belief2human_noemotion = np.array(noemotion["expOrangeNorm"]) 
belief3human_noemotion = np.array(noemotion["expBothNorm"]) 
beliefhuman_noemotion = sort_category([0,1,2],[np.mean(belief1human_noemotion),np.mean(belief2human_noemotion),np.mean(belief3human_noemotion)])

In [6]:
print(worldhuman_emotion)
print(worldhuman_noemotion)
print(beliefhuman_emotion)
print(beliefhuman_noemotion)

[0.152335   0.41017766 0.43748734]
[0.20990514 0.29784484 0.49225001]
[0.40088854 0.29801169 0.30109977]
[0.28662589 0.28182887 0.43154524]


### Means for knowledge and desire inference

In [7]:
knowledgehuman_emotion = (np.array(emotion["knowledge1"])-1)/8
print(np.mean(knowledgehuman_emotion))
desirehuman_emotion = (np.array(emotion["intention3"])-1)/8
print(np.mean(desirehuman_emotion))

0.5296610169491526
0.934322033898305


In [8]:
knowledgehuman_noemotion = (np.array(noemotion["knowledge1"])-1)/8
print(np.mean(knowledgehuman_noemotion))
desirehuman_noemotion = (np.array(noemotion["intention3"])-1)/8
print(np.mean(desirehuman_noemotion))

0.7517857142857143
0.8892857142857142


## Model (for the condition with no emotional display)

In [9]:
# These parameters are fixed across both conditions (emotions and no-emotions)
# A noise parameter which controls how much randomness is added at each conditional sampling (e.g., drawing actions from belief and desire) was allowed to vary across both conditions

# The model assumes that people perceive others to have a knowledgability of around 0.7.
# We simulated this using a beta distribution with a = 5 and b = 2.
a = 5
b = 2

d = .5 # fixed prior for desire
delay = .2 # probability that first action had a delayed effect
human_prior = [0.3816152,0.3063088,0.3120760]

In [10]:
np.random.seed(123456)

In [11]:
with pm.Model() as model1:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=2 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if belief both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange others
    
    outcome2_p = pm.Deterministic("outcome2_p", tt.eq(world,0)*tt.eq(action2,0) + \
                                  tt.eq(world,0)*tt.eq(action1,0)*tt.neq(action2,0)*delay + \
                                  tt.eq(world,1)*tt.eq(action2,1) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*tt.neq(action2,1)*delay + \
                                  tt.eq(world,2)*tt.eq(action1,0)*tt.eq(action2,1)+ \
                                  tt.eq(world,2)*tt.eq(action1,1)*tt.eq(action2,0)) 
    outcome2_p = perturb(outcome2_p, noise)
    outcome2 = pm.Binomial("outcome2", 1, outcome2_p, observed = 1, shape = (1))
    
    happy2 = outcome2*desire*(1-noise) + noise*(1-desire)
    frustrated2 = (1-outcome2)*desire*(1-noise) + noise*(1-desire)
    neutral2 = [1]
    emotion_array2 = tt.transpose(tt.stack(happy2, frustrated2, neutral2 ))
    expression2 = pm.Categorical("expression2",emotion_array2,observed=2 ) # happy, frustrated, neutral
    
    draw = 2000 
    trace1 = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1["world"], return_counts=True)
worldposterior = sort_category(unique, counts)
world_rmse = rmse(worldposterior, worldhuman_noemotion)
print(world_rmse)
unique, counts = np.unique(trace1["belief1"], return_counts=True)
belief1posterior = sort_category(unique, counts)
belief_rmse = rmse(belief1posterior, beliefhuman_noemotion)
print(belief_rmse)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 76 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.14027682241692474
0.18056711834482866


### Summary statistics

In [12]:
unique, counts = np.unique(trace1["world"], return_counts=True)
print("world: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

print("knowledge p: " + str(np.mean(trace1["knowledge_p"])))
unique, counts = np.unique(trace1["knowledge"], return_counts=True)
print("knowledge: " + str(np.mean(trace1["knowledge"])))

unique, counts = np.unique(trace1["belief1"], return_counts=True)
print("belief1: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

unique, counts = np.unique(trace1["belief2"], return_counts=True)
print("belief2: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

unique, counts = np.unique(trace1["desire"], return_counts=True)
print("desire: " + str(np.mean(trace1["desire"])))

world: {0: 220, 1: 2572, 2: 5208}
[0.0275 0.3215 0.651 ]
knowledge p: 0.7250887688272029
knowledge: 0.771125
belief1: {0: 579, 1: 2149, 2: 5272}
[0.072375 0.268625 0.659   ]
belief2: {0: 579, 1: 2149, 2: 5272}
[0.072375 0.268625 0.659   ]
desire: 0.6735


### Confidence intervals

In [51]:
np.random.seed(123456)
print("world :" + str(compute_ci_discrete(flatten(trace1["world"]), HDI = .95, n = 1000, sim = 1000))) #70
print("knowledge :" + str(compute_ci_continuous(flatten(trace1["knowledge_p"]), HDI = .95, n = 70, sim = 1000)))
print("belief1 :" + str(compute_ci_discrete(flatten(trace1["belief1"]), HDI = .95, n =1000, sim = 1000)))
print("belief2 :" + str(compute_ci_discrete(flatten(trace1["belief2"]), HDI = .95, n = 1000, sim = 1000)))
print("desire :" + str(compute_ci_continuous(flatten(trace1["desire"]), HDI = .95, n = 70, sim = 1000)))

world :[array([0.018   , 0.293975, 0.622   ]), array([0.037025, 0.35    , 0.679025])]
knowledge :[0.689338574660182, 0.7614024391721229]
belief1 :[array([0.057, 0.241, 0.63 ]), array([0.089, 0.298, 0.689])]
belief2 :[array([0.057, 0.244, 0.628]), array([0.09 , 0.297, 0.685])]
desire :[0.5571428571428572, 0.7857142857142857]


In [ ]:
az.plot_trace(trace1)

## Model (for the condition with emotional displays)

In [14]:
np.random.seed(123456)

In [15]:
with pm.Model() as model2:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=1 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if belief both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange others
    
    outcome2_p = pm.Deterministic("outcome2_p", tt.eq(world,0)*tt.eq(action2,0) + \
                                  tt.eq(world,0)*tt.eq(action1,0)*tt.neq(action2,0)*delay + \
                                  tt.eq(world,1)*tt.eq(action2,1)+ \
                                  tt.eq(world,1)*tt.eq(action1,1)*tt.neq(action2,1)*delay + \
                                  tt.eq(world,2)*tt.eq(action1,0)*tt.eq(action2,1)+ \
                                  tt.eq(world,2)*tt.eq(action1,1)*tt.eq(action2,0)) 
    outcome2_p = perturb(outcome2_p, noise)
    outcome2 = pm.Binomial("outcome2", 1, outcome2_p, observed = 1, shape = (1))
    
    happy2 = outcome2*desire*(1-noise) + noise*(1-desire)
    frustrated2 = (1-outcome2)*desire*(1-noise) + noise*(1-desire)
    neutral2 = [1]
    emotion_array2 = tt.transpose(tt.stack(happy2, frustrated2, neutral2 ))
    expression2 = pm.Categorical("expression2",emotion_array2,observed=0 ) # happy, frustrated, neutral
    
    draw = 2000 
    trace2 = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace2["world"], return_counts=True)
worldposterior = sort_category(unique, counts)
world_rmse = rmse(worldposterior, worldhuman_emotion)
print(world_rmse)
unique, counts = np.unique(trace2["belief1"], return_counts=True)
belief1posterior = sort_category(unique, counts)
belief_rmse = rmse(belief1posterior, beliefhuman_emotion)
print(belief_rmse)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 77 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.040238991335039054
0.01536544169636947


### Summary statistics

In [16]:
unique, counts = np.unique(trace2["world"], return_counts=True)
print("world: " + str(dict(zip(unique, counts))))
print(sort_category(unique,counts))
print("knowledge p: " + str(np.mean(trace2["knowledge_p"])))
print("knowledge: " + str(np.mean(trace2["knowledge"])))
unique, counts = np.unique(trace2["belief1"], return_counts=True)
print("belief1: " + str(dict(zip(unique, counts))))
print(sort_category(unique,counts))
unique, counts = np.unique(trace2["belief2"], return_counts=True)
print("belief2: " + str(dict(zip(unique, counts))))
print(sort_category(unique,counts))
unique, counts = np.unique(trace2["desire"], return_counts=True)
print("desire: " + str(np.mean(trace2["desire"])))

world: {0: 764, 1: 3489, 2: 3747}
[0.0955   0.436125 0.468375]
knowledge p: 0.6938197847847518
knowledge: 0.527875
belief1: {0: 3360, 1: 2236, 2: 2404}
[0.42   0.2795 0.3005]
belief2: {0: 704, 1: 3475, 2: 3821}
[0.088    0.434375 0.477625]
desire: 0.96125


### Confidence intervals

In [50]:
np.random.seed(123456)
print("world :" + str(compute_ci_discrete(flatten(trace2["world"]), HDI = .95, n = 1000, sim = 1000)))
print("knowledge :" + str(compute_ci_continuous(flatten(trace2["knowledge_p"]), HDI = .95, n = 59, sim = 1000)))
print("belief1 :" + str(compute_ci_discrete(flatten(trace2["belief1"]), HDI = .95, n = 1000, sim = 1000)))
print("belief2 :" + str(compute_ci_discrete(flatten(trace2["belief2"]), HDI = .95, n = 1000, sim = 1000)))
print("desire :" + str(compute_ci_continuous(flatten(trace2["desire"]), HDI = .95, n = 59, sim = 1000)))

world :[array([0.076   , 0.405975, 0.438   ]), array([0.113   , 0.466025, 0.499   ])]
knowledge :[0.652382863412051, 0.7343752532197032]
belief1 :[array([0.391, 0.253, 0.274]), array([0.449   , 0.306025, 0.328   ])]
belief2 :[array([0.071, 0.404, 0.447]), array([0.105   , 0.467025, 0.509   ])]
desire :[0.9148305084745767, 1.0]


In [1]:
az.plot_trace(trace2)

NameError: name 'az' is not defined

# Computational models for tracking study

## I/O preprocessing

In [18]:
tracking_data = pd.read_csv('data/ecl_dat_multiple-cause-tracking.csv', encoding="ISO-8859-1")
emotion_track = tracking_data.loc[tracking_data['condition'] == 'emotion']
noemotion_track = tracking_data.loc[tracking_data['condition'] == 'no-emotion']

### Means for causal inference (at various snapshots)

In [19]:
world1human_emotion_t1 = np.array(emotion_track["relation1BlueNorm"])
world2human_emotion_t1 = np.array(emotion_track["relation1OrangeNorm"])
world3human_emotion_t1 = np.array(emotion_track["relation1OrangeBlueNorm"])
worldhuman_emotion_t1 = sort_category([0,1,2],[np.mean(world1human_emotion_t1),np.mean(world2human_emotion_t1),np.mean(world3human_emotion_t1)])

world1human_noemotion_t1 = np.array(noemotion_track["relation1BlueNorm"]) 
world2human_noemotion_t1 = np.array(noemotion_track["relation1OrangeNorm"]) 
world3human_noemotion_t1 = np.array(noemotion_track["relation1OrangeBlueNorm"])
worldhuman_noemotion_t1 = sort_category([0,1,2],[np.mean(world1human_noemotion_t1),np.mean(world2human_noemotion_t1),np.mean(world3human_noemotion_t1)])

In [20]:
world1human_emotion_t2 = np.array(emotion_track["relation2BlueNorm"])
world2human_emotion_t2 = np.array(emotion_track["relation2OrangeNorm"])
world3human_emotion_t2 = np.array(emotion_track["relation2OrangeBlueNorm"])
worldhuman_emotion_t2 = sort_category([0,1,2],[np.mean(world1human_emotion_t2),np.mean(world2human_emotion_t2),np.mean(world3human_emotion_t2)])

world1human_noemotion_t2 = np.array(noemotion_track["relation2BlueNorm"]) 
world2human_noemotion_t2 = np.array(noemotion_track["relation2OrangeNorm"]) 
world3human_noemotion_t2 = np.array(noemotion_track["relation2OrangeBlueNorm"])
worldhuman_noemotion_t2 = sort_category([0,1,2],[np.mean(world1human_noemotion_t2),np.mean(world2human_noemotion_t2),np.mean(world3human_noemotion_t2)])

In [21]:
world1human_emotion_t3 = np.array(emotion_track["relation3BlueNorm"])
world2human_emotion_t3 = np.array(emotion_track["relation3OrangeNorm"])
world3human_emotion_t3 = np.array(emotion_track["relation3OrangeBlueNorm"])
worldhuman_emotion_t3 = sort_category([0,1,2],[np.mean(world1human_emotion_t3),np.mean(world2human_emotion_t3),np.mean(world3human_emotion_t3)])

world1human_noemotion_t3 = np.array(noemotion_track["relation3BlueNorm"]) 
world2human_noemotion_t3 = np.array(noemotion_track["relation3OrangeNorm"]) 
world3human_noemotion_t3 = np.array(noemotion_track["relation3OrangeBlueNorm"])
worldhuman_noemotion_t3 = sort_category([0,1,2],[np.mean(world1human_noemotion_t3),np.mean(world2human_noemotion_t3),np.mean(world3human_noemotion_t3)])

In [22]:
world1human_emotion_t4 = np.array(emotion_track["relation4BlueNorm"])
world2human_emotion_t4 = np.array(emotion_track["relation4OrangeNorm"])
world3human_emotion_t4 = np.array(emotion_track["relation4OrangeBlueNorm"])
worldhuman_emotion_t4 = sort_category([0,1,2],[np.mean(world1human_emotion_t4),np.mean(world2human_emotion_t4),np.mean(world3human_emotion_t4)])

world1human_noemotion_t4 = np.array(noemotion_track["relation4BlueNorm"]) 
world2human_noemotion_t4 = np.array(noemotion_track["relation4OrangeNorm"]) 
world3human_noemotion_t4 = np.array(noemotion_track["relation4OrangeBlueNorm"])
worldhuman_noemotion_t4 = sort_category([0,1,2],[np.mean(world1human_noemotion_t4),np.mean(world2human_noemotion_t4),np.mean(world3human_noemotion_t4)])

In [23]:
world1human_emotion_t5 = np.array(emotion_track["relation5BlueNorm"])
world2human_emotion_t5 = np.array(emotion_track["relation5OrangeNorm"])
world3human_emotion_t5 = np.array(emotion_track["relation5OrangeBlueNorm"])
worldhuman_emotion_t5 = sort_category([0,1,2],[np.mean(world1human_emotion_t5),np.mean(world2human_emotion_t5),np.mean(world3human_emotion_t5)])

world1human_noemotion_t5 = np.array(noemotion_track["relation5BlueNorm"]) 
world2human_noemotion_t5 = np.array(noemotion_track["relation5OrangeNorm"]) 
world3human_noemotion_t5 = np.array(noemotion_track["relation5OrangeBlueNorm"])
worldhuman_noemotion_t5 = sort_category([0,1,2],[np.mean(world1human_noemotion_t5),np.mean(world2human_noemotion_t5),np.mean(world3human_noemotion_t5)])

## Models (for the condition with no emotional display) across various snapshots

In [24]:
np.random.seed(123456)
with pm.Model() as model1ab:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    world_prior = pm.Categorical("world_prior", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    draw = 2000 
    trace1ab = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1ab["world_prior"], return_counts=True)
worldposterior_t1 = sort_category(unique, counts)
rmse_output_t1 = rmse(worldposterior_t1, worldhuman_noemotion_t1)
print(rmse_output_t1)
unique, counts = np.unique(trace1ab["world"], return_counts=True)
worldposterior_t2 = sort_category(unique, counts)
rmse_output_t2 = rmse(worldposterior_t2, worldhuman_noemotion_t2)
print(rmse_output_t2)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief1_random, world_prior, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 66 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.01131712625682042
0.022070127365948794


In [25]:
np.random.seed(123456)
with pm.Model() as model1c:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=2 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    draw = 2000 
    trace1c = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1c["world"], return_counts=True)
worldposterior_t3 = sort_category(unique, counts)
rmse_output_t3 = rmse(worldposterior_t3, worldhuman_noemotion_t3)
print(rmse_output_t3)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 69 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.1794069406318001


In [26]:
np.random.seed(123456)
with pm.Model() as model1d:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=2 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if belief both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange others
    
    draw = 2000 
    trace1d = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1d["world"], return_counts=True)
worldposterior_t4 = sort_category(unique, counts)
rmse_output_t4 = rmse(worldposterior_t4, worldhuman_noemotion_t4)
print(rmse_output_t4)
unique, counts = np.unique(trace1["world"], return_counts=True)
worldposterior_t5 = sort_category(unique, counts)
rmse_output_t5 = rmse(worldposterior_t5, worldhuman_noemotion_t5)
print(rmse_output_t5)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 75 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.17317066054708688
0.09987023023380058


### Summary statistics

In [27]:
unique, counts = np.unique(trace1ab["world_prior"], return_counts=True)
print("world t1: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t1: {0: 3008, 1: 2481, 2: 2511}
[0.376    0.310125 0.313875]


In [28]:
np.random.seed(123456)
print("world t1 :" + str(compute_ci_discrete(flatten(trace1ab["world_prior"]), HDI = .95, n = 1000, sim = 1000)))

world t1 :[array([0.347, 0.283, 0.285]), array([0.406, 0.336, 0.343])]


In [29]:
unique, counts = np.unique(trace1ab["world"], return_counts=True)
print("world t2: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t2: {0: 3586, 1: 1841, 2: 2573}
[0.44825  0.230125 0.321625]


In [30]:
np.random.seed(123456)
print("world t2 :" + str(compute_ci_discrete(flatten(trace1ab["world"]), HDI = .95, n = 1000, sim = 1000)))

world t2 :[array([0.417975, 0.205   , 0.292   ]), array([0.479, 0.255, 0.352])]


In [31]:
unique, counts = np.unique(trace1c["world"], return_counts=True)
print("world t3: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t3: {0: 1222, 1: 2742, 2: 4036}
[0.15275 0.34275 0.5045 ]


In [32]:
np.random.seed(123456)
print("world t3 :" + str(compute_ci_discrete(flatten(trace1c["world"]), HDI = .95, n = 1000, sim = 1000)))

world t3 :[array([0.13 , 0.312, 0.475]), array([0.177, 0.372, 0.539])]


In [33]:
unique, counts = np.unique(trace1d["world"], return_counts=True)
print("world t4: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t4: {0: 785, 1: 2435, 2: 4780}
[0.098125 0.304375 0.5975  ]


In [34]:
np.random.seed(123456)
print("world t4 :" + str(compute_ci_discrete(flatten(trace1d["world"]), HDI = .95, n = 1000, sim = 1000)))

world t4 :[array([0.079, 0.275, 0.566]), array([0.117, 0.334, 0.629])]


In [35]:
unique, counts = np.unique(trace1["world"], return_counts=True)
print("world t5: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t5: {0: 220, 1: 2572, 2: 5208}
[0.0275 0.3215 0.651 ]


In [36]:
np.random.seed(123456)
print("world t5 :" + str(compute_ci_discrete(flatten(trace1["world"]), HDI = .95, n = 1000, sim = 1000)))

world t5 :[array([0.018   , 0.293975, 0.622   ]), array([0.037025, 0.35    , 0.679025])]


## Models (for the condition with emotional displays) across various snapshots

In [37]:
np.random.seed(123456)
with pm.Model() as model2ab:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    world_prior = pm.Categorical("world_prior", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    draw = 2000 
    trace2ab = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)

unique, counts = np.unique(trace2ab["world_prior"], return_counts=True)
worldposterior_t1 = sort_category(unique, counts)
rmse_output_t1 = rmse(worldposterior_t1, worldhuman_emotion_t1)
print(rmse_output_t1)
unique, counts = np.unique(trace2ab["world"], return_counts=True)
worldposterior_t2 = sort_category(unique, counts)
rmse_output_t2 = rmse(worldposterior_t2, worldhuman_emotion_t2)
print(rmse_output_t2)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief1_random, world_prior, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 82 seconds.
The acceptance probability does not match the target. It is 0.7181823987196119, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


0.003390022173440893
0.02173327744262479


In [38]:
np.random.seed(123456)
with pm.Model() as model2c:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=1 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    draw = 2000 
    trace2c = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)

unique, counts = np.unique(trace2c["world"], return_counts=True)
worldposterior_t3 = sort_category(unique, counts)
rmse_output_t3 = rmse(worldposterior_t3, worldhuman_emotion_t3)
print(rmse_output_t3)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 73 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.045049493459074115


In [39]:
np.random.seed(123456)
with pm.Model() as model2d:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both, others
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=1 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if belief both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange 
    
    draw = 2000 
    trace2d = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace2d["world"], return_counts=True)
worldposterior_t4 = sort_category(unique, counts)
rmse_output_t4 = rmse(worldposterior_t4, worldhuman_emotion_t4)
print(rmse_output_t4)
unique, counts = np.unique(trace2["world"], return_counts=True)
worldposterior_t5 = sort_category(unique, counts)
rmse_output_t5 = rmse(worldposterior_t5, worldhuman_emotion_t5)
print(rmse_output_t5)

Multiprocess sampling (4 chains in 2 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 78 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.046537487425447555
0.0677031243980152


### Summary statistics

In [40]:
unique, counts = np.unique(trace2ab["world_prior"], return_counts=True)
print("world t1: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t1: {0: 3045, 1: 2454, 2: 2501}
[0.380625 0.30675  0.312625]


In [41]:
np.random.seed(123456)
print("world t1 :" + str(compute_ci_discrete(flatten(trace2ab["world_prior"]), HDI = .95, n = 1000, sim = 1000)))

world t1 :[array([0.352   , 0.278   , 0.281975]), array([0.41 , 0.335, 0.341])]


In [42]:
unique, counts = np.unique(trace2ab["world"], return_counts=True)
print("world t2: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t2: {0: 3792, 1: 1675, 2: 2533}
[0.474    0.209375 0.316625]


In [43]:
np.random.seed(123456)
print("world t2 :" + str(compute_ci_discrete(flatten(trace2ab["world"]), HDI = .95, n = 1000, sim = 1000)))

world t2 :[array([0.443, 0.185, 0.289]), array([0.506, 0.235, 0.347])]


In [44]:
unique, counts = np.unique(trace2c["world"], return_counts=True)
print("world t3: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t3: {0: 2223, 1: 2824, 2: 2953}
[0.277875 0.353    0.369125]


In [45]:
np.random.seed(123456)
print("world t3 :" + str(compute_ci_discrete(flatten(trace2c["world"]), HDI = .95, n = 1000, sim = 1000)))

world t3 :[array([0.248975, 0.325   , 0.342   ]), array([0.304, 0.381, 0.399])]


In [46]:
unique, counts = np.unique(trace2d["world"], return_counts=True)
print("world t4: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t4: {0: 2302, 1: 2783, 2: 2915}
[0.28775  0.347875 0.364375]


In [47]:
np.random.seed(123456)
print("world t4 :" + str(compute_ci_discrete(flatten(trace2d["world"]), HDI = .95, n = 1000, sim = 1000)))

world t4 :[array([0.26 , 0.318, 0.335]), array([0.317, 0.377, 0.394])]


In [48]:
unique, counts = np.unique(trace2["world"], return_counts=True)
print("world t5: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t5: {0: 764, 1: 3489, 2: 3747}
[0.0955   0.436125 0.468375]


In [49]:
np.random.seed(123456)
print("world t5 :" + str(compute_ci_discrete(flatten(trace2["world"]), HDI = .95, n = 1000, sim = 1000)))

world t5 :[array([0.076   , 0.405975, 0.438   ]), array([0.113   , 0.466025, 0.499   ])]
